In [ ]:
!pyspark --driver-memory 2g

In [1]:
import os
os.environ.get("SPARK_HOME")

'C:\\Xiaonan\\Courses\\Data engineering\\Spark\\spark-2.2.0-bin-hadoop2.7\\bin'

In [2]:
os.environ.get("HADOOP_HOME")

'C:\\Xiaonan\\Courses\\Data engineering\\Spark\\hadoop-2.7.1'

In [ ]:
import sys
sys.getdefaultencoding()
#reload(sys)
#sys.setdefaultencoding("utf-8")

In [3]:
import findspark
findspark.init()


In [4]:
import configparser
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [5]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']


In [6]:
def create_spark_session():
    """Create spark session.
    Returns:
        spark (SparkSession) - spark session connected to AWS EMR cluster
    """

    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark



In [7]:
def process_song_data(spark, input_data, output_data):
    '''
    Load song data from S3 and extracts the song and artist tables
    
    Keyword arguments:
    spark -- the spark session
    input_data -- the path of input song data from S3
    output_data -- the path of output data to S3
    
    '''
    # get filepath to song data file
    song_data = input_data + "song_data/*/*/*/*json"
    
    #song_data = "./data/song_data/*/*/*/*json"
    
    # read song data file
    df = spark.read.json(song_data).drop_duplicates()
    df.printSchema()

    # extract columns to create songs table
    songs_table = df.select("song_id", "title", "artist_id", "year", "duration").distinct()
    
    # write songs table to parquet files partitioned by year and artist
    songs_table=songs_table.repartition(col("year"),col("artist_id"))
    songs_table.write.parquet(output_data + "songs_table.parquet", mode="overwrite", partitionBy=["year", "artist_id"])

    # extract columns to create artists table
    artists_table = df.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude").distinct()
    
    # write artists table to parquet files
    artists_table.write.parquet(output_data + "artists_table.parquet", mode="overwrite")


In [8]:
def process_log_data(spark, input_data, output_data):
    '''
    Load log data from S3 and extracts the user, time and songplay tables
    
    Keyword arguments:
    spark -- the spark session
    input_data -- the path of input log data from S3
    output_data -- the path of output data to S3
    
    '''
    # get filepath to log data file
    log_data = input_data + "log_data/*/*/*json"

    # read log data file
    df = spark.read.json(log_data).drop_duplicates() 
    df.printSchema()
    
    # filter by actions for song plays
    df = df.filter(df.page=="NextSong")

    # extract columns for users table    
    #user_table = df.select("userId", "firstName", "lastName", "gender", "level").distinct()
    
    #user_table.show(1)
    
    # write users table to parquet files
    # user_table.write.parquet(output_data + "user_table.parquet", mode = "overwrite")

    # create timestamp column from original timestamp column
    #get_timestamp = udf(lambda ts: datetime.datetime.fromtimestamp(int(ts)/1000), T.TimestampType())
    #df = df.withColumn("start_time", get_timestamp(ts)) 
    
    # extract columns to create time table
    #time_table = df.withColumn("hour",     F.hour("start_time"))\
    #                .withColumn("day",     F.dayofmonth("start_time"))\
    #                .withColumn("week",    F.weekofyear("start_time"))\
    #                .withColumn("month",   F.month("start_time"))\
    #                .withColumn("year",    F.year("start_time"))\
    #                .withColumn("weekday", F.dayofweek("start_time"))\
    #                .select("ts","start_time","hour", "day", "week", "month", "year", "weekday")
    
    # write time table to parquet files partitioned by year and month
    #time_table=time_table.repartition(col("year"),col("month"))
    #time_table.write.parquet(output_data + "time_table.parquet", mode="overwrite", partitionBy = ["year", "month"])

    # read in song data to use for songplays table
    #song_df = spark.read.parquet(output_data + "songs_table.parquet")

    # read in artist data to use for songplays table
    #artist_df = spark.read.parquet(output_data + "artist_table.parquet")

    # extract columns from joined song, artist and log datasets to create songplays table 
    #songplays_table = df.selectColumn("songplay_id", F.monotonically_increasing_id())\
    #                    .join(song_df, df.song == song_df.title, how="inner")\
    #                    .select("songplay_id", 
    #                           "start_time", 
    #                           col("userId").alias("user_id"),
    #                            "level", 
    #                            "song_id", 
    #                            "artist_id", 
    #                            col("sessionId").alias("session_id"), 
    #                            "location", 
    #                            col("userAgent").alias("user_agent")) 

    # write songplays table to parquet files partitioned by year and month
    # songplays_table=songplays_table.repartition(col("year"),col("month"))
    # songplays_table.write.parquet(output_data + "songplays_table.parquet", mode = "overwrite", partitionBy = ["year", "month"])


In [9]:
spark = create_spark_session()

In [11]:
sc._conf.get('spark.driver.memory')

NameError: name 'sc' is not defined

In [10]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://udacity-xn/DataEngineering/"

In [ ]:
#process_song_data(spark, input_data, output_data) 

In [ ]:
#song_data = "./data/song_data/*/*/*/*json"
    
# read song data file
#df = spark.read.json(song_data).drop_duplicates()
#df.printSchema()

In [ ]:
#songs_table = df.select("song_id", "title", "artist_id", "year", "duration").distinct()

In [ ]:
#songs_table.show(5)

In [ ]:
#sc=spark.sparkContext
#hadoop_conf=sc._jsc.hadoopConfiguration()

In [ ]:
#hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
#hadoop_conf.set("fs.s3n.awsAccessKeyId", os.environ["AWS_ACCESS_KEY_ID"])
#hadoop_conf.set("fs.s3n.awsSecretAccessKey", os.environ["AWS_SECRET_ACCESS_KEY"])

In [ ]:
#songs_table.write.parquet(output_data + "songs_table.parquet", mode="overwrite", partitionBy=["year", "artist_id"])

In [11]:
# get filepath to log data file
log_data = input_data + "log_data/*/*/*json"

In [12]:
# read log data file
df = spark.read.json(log_data).drop_duplicates() 
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [13]:
# filter by actions for song plays
df = df.filter(df.page=="NextSong")

In [14]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda ts: datetime.fromtimestamp(int(ts)/1000), T.TimestampType())
df = df.withColumn("start_time", get_timestamp("ts")) 

In [23]:
# extract columns to create time table
time_table = df.withColumn("hour",     F.hour("start_time"))\
                .withColumn("day",     F.dayofmonth("start_time"))\
                .withColumn("week",    F.weekofyear("start_time"))\
                .withColumn("month",   F.month("start_time"))\
                .withColumn("year",    F.year("start_time"))\
                .withColumn("weekday", F.dayofweek("start_time"))\
                .select("ts","start_time","hour", "day", "week", "month", "year", "weekday")

In [25]:
time_table.limit(1).toPandas()

,ts,start_time,hour,day,week,month,year,weekday
0,1542296032796,2018-11-15 16:33:52.796,16,15,46,11,2018,5


In [26]:
# write time table to parquet files partitioned by year and month
time_table=time_table.repartition(col("year"),col("month"))

In [27]:
time_table.write.parquet(output_data + "time_table.parquet", mode="overwrite", partitionBy = ["year", "month"])

In [15]:
# read in song data to use for songplays table
song_df = spark.read.parquet(output_data + "songs_table.parquet")

In [16]:
song_df.printSchema()

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- artist_id: string (nullable = true)



In [34]:
df.limit(1).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,start_time
0,Fat Joe,Logged In,Kate,F,21,Harrell,241.34485,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,605,Safe 2 Say [The Incredible] (Album Version - A...,200,1542296032796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,2018-11-15 16:33:52.796


In [17]:
artist_df = spark.read.parquet(output_data + "artists_table.parquet")

In [18]:
artist_df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)



In [19]:
# extract columns from joined song and log datasets to create songplays table 
#songplays_table = df.withColumn("songplay_id", F.monotonically_increasing_id())\
#               .join(song_df, df.song == song_df.title, how="left_outer")\
#               .join(artist_df, df.artist == artist_df.artist_name, how="left_outer")\
#               .select("songplay_id", 
#                      "start_time", 
#                       col("userId").alias("user_id"),
#                        "level", 
#                        "song_id", 
#                        artist_df["artist_id"].alias("artist_id"), 
#                        col("sessionId").alias("session_id"), 
#                        "location", 
#                        col("userAgent").alias("user_agent"),
#                        F.year("start_time").alias("year"),
#                        F.month("start_time").alias("month")

#                      ) 



In [20]:
 df_joined_songs_artists = song_df.join(artist_df, 'artist_id').select("artist_id", "song_id",
                                                                                      "title", "artist_name")

In [21]:
songplays_table = df.withColumn("songplay_id", F.monotonically_increasing_id())\
               .join(df_joined_songs_artists, df.artist == df_joined_songs_artists.artist_name)\
               .select("songplay_id", 
                      "start_time", 
                       col("userId").alias("user_id"),
                        "level", 
                        "song_id", 
                        "artist_id", 
                        col("sessionId").alias("session_id"), 
                        "location", 
                        col("userAgent").alias("user_agent"),
                        F.year("start_time").alias("year"),
                        F.month("start_time").alias("month")

                      ) 

In [22]:
songplays_table.printSchema()

root
 |-- songplay_id: long (nullable = false)
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



In [23]:
# write songplays table to parquet files partitioned by year and month
songplays_table=songplays_table.repartition(col("year"),col("month"))

In [24]:
songplays_table.write.parquet(output_data + "songplays_table.parquet", mode = "overwrite", partitionBy = ["year", "month"])

In [27]:
!pip3 install boto3

  Attempting uninstall: botocore
    Found existing installation: botocore 1.18.16
    Uninstalling botocore-1.18.16:
      Successfully uninstalled botocore-1.18.16
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.3.3
    Uninstalling s3transfer-0.3.3:
      Successfully uninstalled s3transfer-0.3.3


ERROR: awscli 1.18.157 has requirement botocore==1.18.16, but you'll have botocore 1.22.11 which is incompatible.
ERROR: awscli 1.18.157 has requirement s3transfer<0.4.0,>=0.3.0, but you'll have s3transfer 0.5.0 which is incompatible.


In [28]:
import boto3

In [29]:
s3 = boto3.resource('s3',
                       region_name="us-east-1",
                       aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                       aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY']
                   )

In [30]:
sampleDbBucket =  s3.Bucket("udacity-dend")

# TODO: Iterate over bucket objects starting with "ssbgz" and print
for obj in sampleDbBucket.objects.filter(Prefix="log_data"):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-01-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-02-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-03-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-04-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-05-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-06-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-07-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-08-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-09-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log_data/2018/11/2018-11-10-events.json')
s3.ObjectSummary(b

In [74]:
songplays_table.limit(1).toPandas()

,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent,year,month
0,1056561954818,2018-11-21 09:25:43.796,88,paid,SOFNUGC12A6BD530ED,ARN8NCB1187FB49652,744,"Sacramento--Roseville--Arden-Arcade, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",2018,11
